Conections

In [1]:
import pandas as pd
import yaml
from sqlalchemy import create_engine

# cargar configuraciones
with open('../config_fill.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_aw = config['Adventure_Works']
    config_etl = config['ETL_PRO']

url_aw = f"mssql+pyodbc://@{config_aw['host']}/{config_aw['dbname']}?driver={config_aw['driver'].replace(' ', '+')}&trusted_connection={config_aw['trusted_connection']}"
url_etl = f"{config_etl['drivername']}://{config_etl['user']}:{config_etl['password']}@{config_etl['host']}:{config_etl['port']}/{config_etl['dbname']}"

aw_engine = create_engine(url_aw)      # SQL Server (OLTP)
etl_engine = create_engine(url_etl)    # Postgres (DWH)


In [2]:
query_fact_internet = """
SELECT
    sod.SalesOrderID,
    sod.SalesOrderDetailID,
    
    soh.OrderDate,
    soh.DueDate,
    soh.ShipDate,

    soh.TerritoryID,
    soh.CustomerID,

    sod.ProductID,
    sod.OrderQty,
    sod.UnitPrice,
    sod.UnitPriceDiscount,
    sod.LineTotal,

    soh.TaxAmt,
    soh.Freight

FROM Sales.SalesOrderDetail sod
INNER JOIN Sales.SalesOrderHeader soh
    ON sod.SalesOrderID = soh.SalesOrderID

WHERE soh.OnlineOrderFlag = 1;   -- Solo ventas por internet
"""

df_fact = pd.read_sql(query_fact_internet, aw_engine)
df_fact.head(), df_fact.shape


(   SalesOrderID  SalesOrderDetailID  OrderDate    DueDate   ShipDate  \
 0         43697                 353 2011-05-31 2011-06-12 2011-06-07   
 1         43698                 354 2011-05-31 2011-06-12 2011-06-07   
 2         43699                 355 2011-05-31 2011-06-12 2011-06-07   
 3         43700                 356 2011-05-31 2011-06-12 2011-06-07   
 4         43701                 357 2011-05-31 2011-06-12 2011-06-07   
 
    TerritoryID  CustomerID  ProductID  OrderQty  UnitPrice  UnitPriceDiscount  \
 0            6       21768        749         1  3578.2700                0.0   
 1            7       28389        773         1  3399.9900                0.0   
 2            1       25863        773         1  3399.9900                0.0   
 3            4       14501        767         1   699.0982                0.0   
 4            9       11003        773         1  3399.9900                0.0   
 
    LineTotal    TaxAmt  Freight  
 0  3578.2700  286.2616  89.456

In [3]:
dim_product = pd.read_sql("SELECT product_key, product_id FROM dim_product", etl_engine)
dim_customer = pd.read_sql("SELECT customer_key, customer_id FROM dim_customer", etl_engine)
dim_territory = pd.read_sql("SELECT territory_key, territory_id FROM dim_territory", etl_engine)
dim_date = pd.read_sql("SELECT date_key, full_date FROM dim_date", etl_engine)


In [5]:
df_fact.columns


Index(['SalesOrderID', 'SalesOrderDetailID', 'OrderDate', 'DueDate',
       'ShipDate', 'TerritoryID', 'CustomerID', 'ProductID', 'OrderQty',
       'UnitPrice', 'UnitPriceDiscount', 'LineTotal', 'TaxAmt', 'Freight'],
      dtype='object')

In [6]:
df_fact.rename(columns={"ProductID": "product_id"}, inplace=True)


In [7]:
df_fact = df_fact.merge(
    dim_product[["product_key", "product_id"]],
    on="product_id",
    how="left"
)


In [8]:
df_fact[["product_id", "product_key"]].head()


,product_id,product_key
0,749,254
1,773,278
2,773,278
3,767,272
4,773,278


In [14]:
df_fact.rename(columns={"CustomerID": "customer_id"}, inplace=True)


In [15]:
df_fact = df_fact.merge(
    dim_customer[["customer_key", "customer_id"]],
    on="customer_id",
    how="left"
)


In [16]:
df_fact[["customer_id", "customer_key"]].head()


,customer_id,customer_key
0,21768,17487
1,28389,9723
2,25863,17990
3,14501,12131
4,11003,19056


In [17]:
df_fact.rename(columns={"TerritoryID": "territory_id"}, inplace=True)
df_fact = df_fact.merge(dim_territory[["territory_key", "territory_id"]], on="territory_id", how="left")


In [18]:
df_fact = df_fact.merge(dim_date[['date_key', 'full_date']],
                        left_on='OrderDate', right_on='full_date',
                        how='left') \
                 .rename(columns={'date_key': 'order_date_key'}) \
                 .drop(columns=['full_date'])

df_fact = df_fact.merge(dim_date[['date_key', 'full_date']],
                        left_on='DueDate', right_on='full_date',
                        how='left') \
                 .rename(columns={'date_key': 'due_date_key'}) \
                 .drop(columns=['full_date'])

df_fact = df_fact.merge(dim_date[['date_key', 'full_date']],
                        left_on='ShipDate', right_on='full_date',
                        how='left') \
                 .rename(columns={'date_key': 'ship_date_key'}) \
                 .drop(columns=['full_date'])


In [19]:
df_fact["discount_amount"] = df_fact["UnitPrice"] * df_fact["UnitPriceDiscount"]
df_fact["gross_sales"] = df_fact["UnitPrice"] * df_fact["OrderQty"]
df_fact["net_sales"] = df_fact["gross_sales"] - df_fact["discount_amount"]


In [20]:
fact_internet = df_fact[[
    "SalesOrderID",
    "SalesOrderDetailID",
    
    "order_date_key",
    "due_date_key",
    "ship_date_key",

    "customer_key",
    "product_key",
    "territory_key",

    "OrderQty",
    "UnitPrice",
    "UnitPriceDiscount",
    "discount_amount",
    "gross_sales",
    "net_sales",
    "LineTotal",
    "TaxAmt",
    "Freight"
]]


In [21]:
fact_internet.to_sql(
    "fact_internet_sales",
    etl_engine,
    if_exists="replace",
    index=False
)


398